In [5]:
from pyspark.sql import SparkSession

sp = SparkSession.builder.appName('a').getOrCreate()
data = sp.read.option('inferSchema', True).option('header', False).csv('corrected.csv')
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: d

In [6]:
colnames = [ "duration", "protocol_type", "service", "flag",
 "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
 "hot", "num_failed_logins", "logged_in", "num_compromised",
 "root_shell", "su_attempted", "num_root", "num_file_creations",
 "num_shells", "num_access_files", "num_outbound_cmds",
 "is_host_login", "is_guest_login", "count", "srv_count",
 "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
 "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
 "dst_host_count", "dst_host_srv_count",
 "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
 "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
 "dst_host_serror_rate", "dst_host_srv_serror_rate",
 "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
 "label"]
data = data.toDF(*colnames)

In [7]:
data.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true

In [13]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

def oneHotPipeline(inputcol):
    indexer = StringIndexer(inputCol = inputcol, outputCol = inputcol + 'index')
    encoder = OneHotEncoder(inputCol = inputcol + 'index', outputCol = inputcol + '_vec')
    pipe = Pipeline(stages = [indexer, encoder])
    return pipe, inputcol + '_vec'

def clustering_score(data, k):
    proto_pipe, proto = oneHotPipeline('protocol_type')
    serv_pipe, serv = oneHotPipeline('service')
    flag_pipe, flag_vec = oneHotPipeline('flag')

    cols = set(data.columns) - set(['label', 'protocol_type', 'service', 'flag'])
    cols.add(proto)
    cols.add(serv)
    cols.add(flag_vec)
    cols = list(cols)
    assembler = VectorAssembler(inputCols = cols, outputCol = 'features')
    scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledfeatures', withStd = True, withMean = False)
    kmeans = KMeans(seed = 1234, k = k, maxIter = 20, tol = 1.0e-5, predictionCol = 'cluster', featuresCol = 'scaledfeatures')
    pipe = Pipeline(stages = [proto_pipe, serv_pipe, flag_pipe, assembler, scaler, kmeans])
    model = pipe.fit(data)
    kmodel = model.stages[-1]
    return kmodel.summary.trainingCost

for k in range(60, 271, 30):
    print(k, clustering_score(data, k))

Exception ignored in: <function JavaWrapper.__del__ at 0x7f958734d510>
Traceback (most recent call last):
  File "/home/linux/.local/lib/python3.10/site-packages/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'KMeans' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x7f958734d510>
Traceback (most recent call last):
  File "/home/linux/.local/lib/python3.10/site-packages/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'KMeans' object has no attribute '_java_obj'
24/11/04 00:11:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/11/04 00:11:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


60 10029672.07907319


90 4095096.839767554


120 1077646.9729078836


150 605132.6617829703


180 480236.3997702935


210 371392.7738260246


240 282595.5268774598


270 243817.10598017703
